In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
# from config_profile import args
# from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from scipy.optimize import linear_sum_assignment
import sys
import matplotlib.pyplot as plt

<span style="color:blue">Read in features</span>

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

q = torch.load("./keypoint_descriptor/p_description.pt")  # torch.Size([5, 100, 128])
d = torch.load("./keypoint_descriptor/q_description.pt")   # torch.Size([5, 100, 128])

print(q.shape)
print(d.shape)

torch.Size([5, 100, 128])
torch.Size([5, 100, 128])


<span style="color:blue">Similarity Matrix - one-to-one & many-to-many</span>

In [3]:
data = np.zeros((100, 100))
similarity_list_one = np.zeros((5, 5))
similarity_list_many = np.zeros((5, 5))
cost_matrix = torch.tensor(data)

for i in range(len(q)):                 # 5
    for j in range(len(d)):             # 5  
        s_one = 0
        s_many = 0
        for k in range(len(q[0])):      # 100
            for l in range(len(d[0])):  # 100
                cost_matrix[k][l] = torch.dist(q[i][k], d[j][l], p=2)
                
            print("\r{},{},{}".format(i+1, j+1, k+1), end='')
            sys.stdout.flush()
        
        # one2one
        cost_matrix_np = cost_matrix.detach().numpy()
        row_ind, col_ind = linear_sum_assignment(cost_matrix_np)   # Hungarian
        
        for m in range(len(cost_matrix_np[0])):       # 100
            s_one += np.exp(-cost_matrix_np[m][col_ind[m]])
        similarity_list_one[i][j] = s_one


5,5,100

In [6]:
print(col_ind)

[ 2  0  1  4  3  6  5  8 11 10  7 13  9 64 15 19 21 83 71 43 59 23 24 73
 53 51 25 22 12 34 44 63 16 41 52 30 32 92 48 99 58 37 18 27 26 35 14 46
 61 97 79 31 29 39 36 38 40 86 70 88 47 42 72 87 84 89 55 62 50 49 54 60
 85 78 67 17 77 76 33 68 69 45 96 94 65 91 75 74 98 81 82 66 20 28 80 57
 56 93 90 95]


<span style="color:blue">Similarity Matrix - many-to-many</span>

In [4]:
print(similarity_list_one.shape)
print(similarity_list_one[0])


(5, 5)
[52.48320991 34.67443389 34.15876008 32.77626014 35.50153326]
(5, 5)
[7644.39947098 7212.65969632 7279.61619272 7061.63438407 7235.37570869]


<span style="color:blue">Save similarity matrix</span>

In [7]:
similarity_list_one
similarity_list_many
matrix_one = torch.from_numpy(similarity_list_one)
matrix_many = torch.from_numpy(similarity_list_many)

torch.save(matrix_one, "similarity_one.pt")
torch.save(matrix_many, "similarity_many.pt")
print("similarity matrix one: ", matrix_one.shape)
print("similarity matrix many: ", matrix_many.shape)

similarity matrix one:  torch.Size([35, 140])
similarity matrix many:  torch.Size([35, 140])
